In [1]:
!mkdir -p './annual-reports'
# !wget 'https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q4/filing/10-Q4-2024-As-Filed.pdf' -O './annual-reports/dsk-Q4.pdf'
!wget 'https://s21.q4cdn.com/399680738/files/doc_news/Meta-Reports-Second-Quarter-2024-Results-2024.pdf' -O './annual-reports/meta-2024.pdf'

--2025-01-11 16:03:07--  https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q4/filing/10-Q4-2024-As-Filed.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.207.2, 68.70.207.1, 68.70.207.3, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.207.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 963934 (941K) [application/pdf]
Saving to: ‘./annual-reports/dsk-Q4.pdf’

./annual-reports/ds 100%[===================>] 941.34K  --.-KB/s    in 0.1s    

2025-01-11 16:03:07 (8.30 MB/s) - ‘./annual-reports/dsk-Q4.pdf’ saved [963934/963934]

--2025-01-11 16:03:07--  https://s21.q4cdn.com/399680738/files/doc_news/Meta-Reports-Second-Quarter-2024-Results-2024.pdf
Resolving s21.q4cdn.com (s21.q4cdn.com)... 68.70.207.2, 68.70.207.1, 68.70.207.3, ...
Connecting to s21.q4cdn.com (s21.q4cdn.com)|68.70.207.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195613 (191K) [application/pdf]
Saving to: ‘./annual-reports/meta-2024.pdf’

./annual-reports/

In [2]:
import os

os.environ['UNSTRUCTURED_API_KEY'] = 'Z7XDM1JJUs9w78saUYwHiAhj7ZVVnX'
os.environ['UNSTRUCTURED_API_URL'] = 'https://api.unstructured.io/general/v0/general'

In [3]:
os.getenv('UNSTRUCTURED_API_KEY'), os.getenv('UNSTRUCTURED_API_URL')

('Z7XDM1JJUs9w78saUYwHiAhj7ZVVnX',
 'https://api.unstructured.io/general/v0/general')

In [38]:
# Parse unstructured data via api, limit of 1000 pages per month
import os

from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
    LocalIndexerConfig,
    LocalDownloaderConfig,
    LocalConnectionConfig,
    LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig

FILENAME = 'meta-2024.pdf'
INPUT_PATH = './annual-reports/' + FILENAME
OUTPUT_PATH = './parsed_data/'

if FILENAME + '.json' not in os.listdir(OUTPUT_PATH):
    Pipeline.from_configs(
            context=ProcessorConfig(),
            indexer_config=LocalIndexerConfig(input_path=INPUT_PATH),
            downloader_config=LocalDownloaderConfig(),
            source_connection_config=LocalConnectionConfig(),
            partitioner_config=PartitionerConfig(
                partition_by_api=True,
                api_key=os.getenv("UNSTRUCTURED_API_KEY"),
                partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
                strategy="hi_res",
                additional_partition_args={
                    "split_pdf_page": True,
                    "split_pdf_allow_failed": True,
                    "split_pdf_concurrency_level": 15
                }
            ),
            uploader_config=LocalUploaderConfig(output_dir=OUTPUT_PATH)
        ).run()

In [7]:
from collections import Counter
import json
with open(os.path.join(OUTPUT_PATH, 'meta-2024.pdf.json'), 'r') as f:
    elements = json.load(f)

display(Counter(element.get('type') for element in elements))

Counter({'NarrativeText': 58, 'Title': 19, 'Table': 11})

In [22]:
for element in elements:
  if element['type'] == 'Table':
    print(element.get('text', ''))

Three Months Ended June 30, In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 39,071 $ 31,999 22 % Costs and expenses 24,224 22,607 7 % Income from operations $ 14,847 $ 9,392 58 % Operating margin 38 % 29 % Provision for income taxes $ 1,641 $ 1,505 9 % Effective tax rate 11 % 16 % Net income $ 13,465 $ 7,788 73 % Diluted earnings per share (EPS) $ 5.16 $ 2.98 73 %
Three Months Ended June 30, Six Months Ended June 30, 2024 2023 2024 2023 Revenue $ 39,071 $ 31,999 $ 75,527 $ 60,645 Costs and expenses: Cost of revenue 7,308 5,945 13,948 12,054 Research and development 10,537 9,344 20,515 18,725 Marketing and sales 2,721 3,154 5,285 6,198 General and administrative (1) 3,658 4,164 7,114 7,049 Total costs and expenses 24,224 22,607 46,862 44,026 Income from operations 14,847 9,392 28,665 16,619 Interest and other income (expense), net 259 (99) 624 (19) Income before provision for income taxes 15,106 9,293 29,289 16,600 Provision for income taxes 1,641 1,50

In [32]:
from openai import OpenAI
import fitz
from tqdm import tqdm

# Initialize the OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_table_description(table_content, document_context):
    prompt = f"""
    Given the following table and its context from the original document,
    provide a detailed description of the table. Then, include the table in markdown format.

    Original Document Context:
    {document_context}

    Table Content:
    {table_content}

    Please provide:
    1. A comprehensive description of the table.
    2. The table in markdown format.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that describes tables and formats them in markdown."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def extract_text_from_pdf(pdf_path):
    text = ""

    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

document_content = extract_text_from_pdf(INPUT_PATH)

# Process each table in the directory
for element in tqdm(elements):
  if element.get('type') == 'Table':
    table_content = element.get('text')

    # Get description and markdown table from GPT-4
    result = get_table_description(table_content, document_content)
    element['text'] = result

print("Processing complete.")


  8%|▊         | 7/88 [00:09<01:44,  1.29s/it]

### Detailed Description of the Table

The table provides a summary of Meta Platforms, Inc.'s financial performance for the second quarter ending June 30, 2024, compared to the same period in 2023. 

- **Revenue**: The company reported a revenue of $39.071 billion in Q2 2024, which is a 22% increase from the $31.999 billion reported in Q2 2023.
- **Costs and Expenses**: Total costs and expenses rose by 7%, from $22.607 billion in Q2 2023 to $24.224 billion in Q2 2024.
- **Income from Operations**: Income from operations saw a significant increase of 58%, climbing from $9.392 billion in 2023 to $14.847 billion in 2024.
- **Operating Margin**: The operating margin increased from 29% in Q2 2023 to 38% in Q2 2024.
- **Provision for Income Taxes**: The provision for income taxes increased by 9%, from $1.505 billion in 2023 to $1.641 billion in 2024.
- **Effective Tax Rate**: The effective tax rate lowered from 16% in 2023 to 11% in 2024.
- **Net Income**: Net income nearly doubled, increasi

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 62%|██████▎   | 55/88 [00:22<00:12,  2.72it/s]

### Description of the Table

The table presents Meta Platforms, Inc.'s condensed consolidated statements of income for the second quarter, as well as the first half of the fiscal year 2024, compared to the same periods in 2023. The financial figures are in millions of dollars, except for per-share amounts. The table illustrates several key financial data points:

- **Revenue:** Total revenue increased significantly, reaching $39,071 million for the three months ended June 30, 2024, compared to $31,999 million in 2023. For the six-month period, revenue grew from $60,645 million in 2023 to $75,527 million in 2024.

- **Costs and Expenses:** These are divided into cost of revenue, research and development, marketing and sales, and general and administrative expenses. Each category shows the expenditures for both quarters and six-month periods across 2023 and 2024. Notably, research and development saw a noticeable increase, reflecting Meta’s investment in new technologies.

- **Income fr

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 64%|██████▎   | 56/88 [00:29<00:17,  1.86it/s]

...

### Description of the Table

The table presents condensed consolidated statements of income for Meta Platforms, Inc. for the three- and six-month periods ending June 30 for the years 2024 and 2023. It is a financial summary that provides key figures pertaining to revenue, costs and expenses, and net income. The table shows both quarterly and half-yearly comparisons for each line item, illustrating the financial performance of the company over these periods. Key components of the table include:

1. **Revenue**: Indicating the total income generated from operational activities.
2. **Costs and Expenses**: Includes distinct accounts such as cost of revenue, research and development, marketing and sales, and general and administrative expenses.
3. **Income from Operations**: Reflecting the profit after operational costs have been deducted from total revenue.
4. **Income before Provision for Income Taxes**: Calculated after accounting for interest and other income/expenses.
5. **Provision f

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
 93%|█████████▎| 82/88 [01:56<00:14,  2.44s/it]

### Description of the Table

The table presents financial results for Meta Platforms, Inc. for the three and six months ended June 30, 2024, and June 30, 2023. The table is divided into revenue and income (loss) from operations across two main segments: Family of Apps and Reality Labs.

- **Revenue Section**:
  - **Advertising**: Shows the revenue generated from advertising activities, which increased from $31,498 million in Q2 2023 to $38,329 million in Q2 2024, and from $59,599 million for the six months ended June 30, 2023, to $73,965 million for the same period in 2024.
  - **Other Revenue**: Includes revenue from sources other than advertising, which also increased year-over-year for both three-month and six-month periods.
  - **Family of Apps**: Combines both advertising and other revenue related to Meta's suite of applications. The increase in revenue is noticeable from $31,723 million in Q2 2023 to $38,718 million in Q2 2024.
  - **Reality Labs**: Accounts for revenue from vir

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 88/88 [02:09<00:00,  1.48s/it]

### Description of the Table

The provided table details key financial metrics for Meta Platforms, Inc., highlighting both the second quarter and first half-year periods ending June 30 for the years 2023 and 2024. The table compares GAAP (Generally Accepted Accounting Principles) revenue, the foreign exchange impact, and adjusted revenue figures for both total revenue and advertising revenue. It also includes year-over-year percentage changes for these metrics. Furthermore, it presents figures for operating cash flows, capital expenditures, principal lease payments, and free cash flow, comparing these results between the two years.

Here's a breakdown of the table content:

1. **Revenue Metrics:**
   - **GAAP Revenue:** The unadjusted financial performance measure, which shows significant growth from 2023 to 2024 for both quarterly and half-year periods.
   - **Foreign Exchange Effect:** Accounts for revenue changes due to fluctuations in exchange rates, allowing for the exclusion of t

In [39]:
# Save the updated elements to a new file
PREFIX = 'processed-'
with open(os.path.join(OUTPUT_PATH, PREFIX + FILENAME + '.json'), 'w') as f:
    json.dump(elements, f, indent=4)

# Use DSPy to create simple RAG

In [40]:
import dspy

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

In [42]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

elements = json.load(open(OUTPUT_PATH + PREFIX + FILENAME + '.json'))
corpus = [element['text'][:max_characters] for element in elements]
print(f"Loaded {len(corpus)} documents. Will encode them below.")
embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

Loaded 88 documents. Will encode them below.


In [45]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [46]:
rag = RAG()
rag(question="What is meta's revenue?")

16:52:16 - LiteLLM:INFO: utils.py:2741 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:52:18 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
INFO: Wrapper: Completed Call, calling success_handler


Prediction(
    reasoning="Meta's revenue for the three months ended June 30, 2024, is reported as $39,071 million, which shows a significant increase from $31,999 million in the same period of 2023. For the six-month period, revenue grew from $60,645 million in 2023 to $75,527 million in 2024. This indicates strong financial performance and growth in revenue.",
    response="Meta's revenue for the three months ended June 30, 2024, is $39,071 million, and for the six months ended June 30, 2024, it is $75,527 million."
)